# This notebook can be used to query NIC for fast ice data
The goal is to use the functions herewith to query for fast ice data, and optionally to append the results to the existing version of the covariate grid.  
  
In this example, we use it to add covariates of ice stability, persistence, and predictability: 
  * stability is here defined as fast ice being present in the grid cell by end of December of target year (2011)
  * persistence is here defined as fast ice being present in the grid cell by end of February of 1 or 2 (or both) years prior
  * predictability is here defined as fast ice being present in the grid cell by end of December of anyone of past 5 years

In [1]:
## Loading the libraries
list.of.packages <- c("rgdal", "proj4","rgeos","maptools","raster","stringr","plyr","dplyr","xml2","httr","ggplot2","data.table")
lapply(list.of.packages, require, character.only = TRUE)


Loading required package: rgdal

Loading required package: sp

rgdal: version: 1.2-20, (SVN revision 725)
 Geospatial Data Abstraction Library extensions to R successfully loaded
 Loaded GDAL runtime: GDAL 2.2.3, released 2017/11/20
 Path to GDAL shared files: /usr/share/gdal/2.2
 GDAL binary built with GEOS: TRUE 
 Loaded PROJ.4 runtime: Rel. 4.9.3, 15 August 2016, [PJ_VERSION: 493]
 Path to PROJ.4 shared files: (autodetected)
 Linking to sp version: 1.4-1 

Loading required package: proj4


Attaching package: ‘proj4’


The following object is masked from ‘package:rgdal’:

    project


Loading required package: rgeos

rgeos version: 0.5-2, (SVN revision 621)
 GEOS runtime version: 3.6.2-CAPI-1.10.2 
 Linking to sp version: 1.4-1 
 Polygon checking: TRUE 


Loading required package: maptools

Checking rgeos availability: TRUE

Loading required package: raster

Loading required package: stringr

Loading required package: plyr

Loading required package: dplyr


Attaching package: ‘dplyr

[[1]]
[1] TRUE

[[2]]
[1] TRUE

[[3]]
[1] TRUE

[[4]]
[1] TRUE

[[5]]
[1] TRUE

[[6]]
[1] TRUE

[[7]]
[1] TRUE

[[8]]
[1] TRUE

[[9]]
[1] TRUE

[[10]]
[1] TRUE

[[11]]
[1] TRUE

[[12]]
[1] TRUE

In [77]:
## Setting needed directories
pathToGit<-"/home/ubuntu/fastIce/FastIceCovars/"
nicsavedir<-"/home/ubuntu/fastIce/nictemp/"

## Sourcing the utilities file
source(paste0(pathToGit,"scripts/fastIceCovars_utils.R"))

## Load the spatialpoints data.frame
load(paste0(pathToGit,"data/studyarea_points_wNearLand.RData"))

In [70]:
## Attribute with ice stability, persistence, and predictability
sspdata<-getFastIceSPPdata(spdf=studyarea_pointswLand,iceyear=2011,iceareas="")


[1] "processing single fast ice date"
OGR data source with driver: ESRI Shapefile 
Source: "/home/ubuntu/fastIce/nictemp/antarc111226", layer: "ANTARC111226"
with 393 features
It has 18 fields
[1] "processing single fast ice date"
OGR data source with driver: ESRI Shapefile 
Source: "/home/ubuntu/fastIce/nictemp/antarc100222", layer: "ANTARC100222"
with 371 features
It has 18 fields
[1] "processing single fast ice date"
OGR data source with driver: ESRI Shapefile 
Source: "/home/ubuntu/fastIce/nictemp/antarc090223", layer: "ANTARC090223"
with 398 features
It has 18 fields
[1] "processing single fast ice date"
OGR data source with driver: ESRI Shapefile 
Source: "/home/ubuntu/fastIce/nictemp/antarc101227", layer: "ANTARC101227"
with 350 features
It has 18 fields
[1] "processing single fast ice date"
OGR data source with driver: ESRI Shapefile 
Source: "/home/ubuntu/fastIce/nictemp/antarc091228", layer: "ANTARC091228"
with 372 features
It has 18 fields
[1] "processing single fast ice dat

In [78]:
head(sspdata);nrow(sspdata)==nrow(studyarea_pointswLand)

,pointId,DecemberIcePresence,Persistence2Years,Dec1yP,Dec2yP,Dec3yP,Dec4yP,Dec5yP,PredictabilityDec5Years
,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1021873,0,0,0,0,0,0,0,0
2,1021874,0,0,0,0,0,0,0,0
3,1021875,0,0,0,0,0,0,0,0
4,1021876,0,0,0,0,0,0,0,0
5,1021877,0,0,0,0,0,0,0,0
6,1021878,0,0,0,0,0,0,0,0


[1] TRUE

In [79]:
## Merging with the spatial.points data.frame, and saving it
studyarea_pointswLand<-merge(studyarea_pointswLand,sspdata,by.x="pointid",by.y="pointId")
head(studyarea_pointswLand);nrow(studyarea_pointswLand)==nrow(sspdata)

,pointid,meanslope,meanbathy,slope,bathy,shoredist,glacierdist,nearLineId,near_x,near_y,⋯,cont300dist,cont800dist,DecemberIcePresence,Persistence2Years,Dec1yP,Dec2yP,Dec3yP,Dec4yP,Dec5yP,PredictabilityDec5Years
,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
187706,68791,2.621992,-2846.908,2.5563672,-2864.911,206728.3,247849.6,2351,-2273383,2327082,⋯,142791.6,588996.1,0,0,0,0,0,0,0,0
187711,68792,4.591249,-2587.330,4.4836812,-2612.724,208790.2,250334.0,2351,-2273383,2327082,⋯,142791.6,588996.1,0,0,0,0,0,0,0,0
187798,68819,1.411659,-3014.080,1.5170590,-3024.577,200154.1,241050.2,2351,-2273383,2327082,⋯,138016.9,584654.8,0,0,0,0,0,0,0,0
187799,68820,1.413779,-2920.630,0.7565953,-2930.036,202161.9,243501.3,2351,-2273383,2327082,⋯,138016.9,584654.8,0,0,0,0,0,0,0,0
187800,68821,4.561732,-2686.220,4.5623317,-2716.875,204263.8,246029.6,2351,-2273383,2327082,⋯,138016.9,584654.8,0,0,0,0,0,0,0,0
187801,68822,4.902608,-2264.930,5.1146288,-2277.257,206460.4,248632.8,2351,-2273383,2327082,⋯,138016.9,584654.8,0,0,0,0,0,0,0,0


[1] TRUE

In [81]:
save(studyarea_pointswLand, file=paste0(pathToGit,"data/studyarea_points_wNearLand.RData"))